In [1]:
# Import standard libraries
import os
from contextlib import redirect_stdout

import sys
# append coeqwal packages to path
sys.path.append('./coeqwalpackage')

import numpy as np
import pandas as pd
import datetime as dt


In [2]:
# Import custom libraries
# Note: on my computer the next import doesn't work the first time I call it, why? If I re-run the cell, then it is ok. MUST DEBUG
from coeqwalpackage.metrics import *
import cqwlutils as cu
import plotting as pu

F:\Dino\COEQWAL_GIT\coeqwal\notebooks\coeqwalpackage\cqwlutils.py:17: SyntaxWarning: invalid escape sequence '\d'
  topsplit = re.split('(\d+)', topleft)
F:\Dino\COEQWAL_GIT\coeqwal\notebooks\coeqwalpackage\cqwlutils.py:18: SyntaxWarning: invalid escape sequence '\d'
  botsplit = re.split('(\d+)', bottomright)


## Define contol file name

In [4]:
CtrlFile = 'CalSim3DataExtractionInitFile_v3.xlsx'
CtrlTab = 'Init'

## Read from control file

In [6]:
ScenarioListFile, ScenarioListTab, ScenarioListPath, DVDssNamesOutPath, SVDssNamesOutPath, ScenarioIndicesOutPath, DssDirsOutPath, VarListPath, VarListFile, VarListTab, VarOutPath, DataOutPath, ConvertDataOutPath, ExtractionSubPath, DemandDeliverySubPath, ModelSubPath, GroupDataDirPath, ScenarioDir, DVDssMin, DVDssMax, SVDssMin, SVDssMax, NameMin, NameMax, DirMin, DirMax, IndexMin, IndexMax, StartMin, StartMax, EndMin, EndMax, VarMin, VarMax, DemandFilePath, DemandFileName, DemandFileTab, DemMin, DemMax = cu.read_init_file(CtrlFile, CtrlTab)

### Read in Data

In [8]:
df, dss_names = read_in_df(ConvertDataOutPath,DVDssNamesOutPath)

In [9]:
df = add_water_year_column(df)

F:\Dino\COEQWAL_GIT\coeqwal\notebooks\coeqwalpackage\metrics.py:175: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  return df_copy.drop(["Date", "Year", "Month"], axis=1)


### Get metrics paths and make directories if needed

In [11]:
metrics_path = GroupDataDirPath + "/metrics_output"
if not os.path.exists(metrics_path):
    os.makedirs(metrics_path)

## Metrics Examples

In [13]:
# To get a single metric and variable dataframe, type metrics[Month_{var}_{metric}] ie metrics['Apr_S_RESTOT_NOD_mnth_avg'], thresholds["S_SHSTA_frequency_hitting_level5"]
# Month = Apr, Sept... or None
# Metric = [ann_avg, mnth_avg, compute_iqr, ann_percentile, moy_avgs, mnth_percentile, annual_totals]

In [14]:
# Thresholds

variables = ["S_SHSTA_", "S_OROVL_", "S_TRNTY_", "S_SLUIS_CVP_", "S_SLUIS_SWP_"]
variables_level1 = ["S_SHSTALEVEL1DV", "S_OROVLLEVEL1DV", "S_TRNTYLEVEL1DV", "S_SLUIS_CVPLEVEL1DV", "S_SLUIS_SWPLEVEL1DV"]
variables_level5 = ["S_SHSTALEVEL5DV", "S_OROVLLEVEL5DV", "S_TRNTYLEVEL5DV", "S_SLUIS_CVPLEVEL5DV", "S_SLUIS_SWPLEVEL5DV"]
write_out_dfs = []

In [15]:
thresholds = {}

for var, var_lvl1, var_lvl5 in zip(variables, variables_level1, variables_level5):
    # LEVEL5DV calculation (LEVEL5DV - var < 0)
    exceedance_days, exceedance_days_fraction = frequency_hitting_level(df, dss_names, var, var_lvl5, "TAF", f"All_Prob_{var}flood")
    thresholds[f"{var}frequency_hitting_level5"] = (exceedance_days, exceedance_days_fraction)

    # LEVEL1DV calculation (var - LEVEL1DV < 0)
    exceedance_days, exceedance_days_fraction = frequency_hitting_level(df, dss_names, var_lvl1, var, "TAF", f"All_Prob_{var}dead")
    thresholds[f"{var}frequency_hitting_level1"] = (exceedance_days, exceedance_days_fraction)

    # September LEVEL1DV calculation
    exceedance_days, exceedance_days_fraction = frequency_hitting_level(df, dss_names, var_lvl1, var, "TAF", f"Sept_Prob_{var}dead", months=[9])
    thresholds[f"Sept_{var}frequency_hitting_level1"] = (exceedance_days, exceedance_days_fraction)

for threshold, (threshold_df, threshold_fraction) in thresholds.items():
    print(f"Threshold: {threshold}")
    #display(threshold_df)
    write_out_dfs.append(threshold_fraction)
    display(threshold_fraction)

Threshold: S_SHSTA_frequency_hitting_level5


,All_Prob_S_SHSTA_flood
s0001,0.333333
s0002,0.315000
s0003,0.222500
s0004,0.192500
s0005,0.175833
s0006,0.303333
s0007,0.301667
s0008,0.300833
s0009,0.300833
s0010,0.301667


Threshold: S_SHSTA_frequency_hitting_level1


,All_Prob_S_SHSTA_dead
s0001,0.004167
s0002,0.017500
s0003,0.038333
s0004,0.052500
s0005,0.053333
s0006,0.003333
s0007,0.000000
s0008,0.000000
s0009,0.000000
s0010,0.000000


Threshold: Sept_S_SHSTA_frequency_hitting_level1


...

,All_Prob_S_SLUIS_SWP_flood
s0001,0.065833
s0002,0.073333
s0003,0.060000
s0004,0.051667
s0005,0.042500
s0006,0.057500
s0007,0.160000
s0008,0.180000
s0009,0.189167
s0010,0.190000


Threshold: S_SLUIS_SWP_frequency_hitting_level1


,All_Prob_S_SLUIS_SWP_dead
s0001,0.029167
s0002,0.026667
s0003,0.027500
s0004,0.055833
s0005,0.043333
s0006,0.040000
s0007,0.030833
s0008,0.033333
s0009,0.033333
s0010,0.033333


Threshold: Sept_S_SLUIS_SWP_frequency_hitting_level1


,Sept_Prob_S_SLUIS_SWP_dead
s0001,0.05
s0002,0.05
s0003,0.06
s0004,0.13
s0005,0.07
s0006,0.06
s0007,0.06
s0008,0.05
s0009,0.07
s0010,0.07


In [16]:
# Other Metrics - not following excel file correctly

variables = [
    "S_RESTOT_NOD_",
    "S_SHSTA_", "S_OROVL_", "S_TRNTY_", "S_MELON_", "S_MLRTN_",
    "DEL_SWP_TOTAL_", "DEL_SWP_PMI_", "DEL_SWP_PAG_", "DEL_CVP_TOTAL_", "DEL_CVP_PAG_TOTAL_", "DEL_CVP_PSCEX_TOTAL_", "DEL_CVP_PRF_TOTAL_",
    "D_TOTAL_", "NDO_",
    "X2_PRV_KM_",
    "DEL_CVP_PSC_N_", "DEL_CVP_PAG_N_", "DEL_CVP_PAG_S_", "DEL_CVP_PRF_S_", "DEL_CVP_PMI_N_", "DEL_CVP_PMI_S_"
] 

In [17]:
metrics = {}

for var in variables:
    if var in ["C_SAC041_", "C_SJR070_", "C_SAC000_", "C_SJR070_", "C_DMC000_TD_", "C_CAA003_TD_", "NDO_"]:
        units = "CFS"
    elif var == "X2_PRV_KM_":
        units = "KM"
    else:
        units = "TAF"

    if var in ["S_RESTOT_NOD_"]: ## What's the difference between this and NOD_STORAGE?
        metrics[f"Apr_{var}mnth_avg"] = mnth_avg(df, dss_names, var, 4, units) 
        metrics[f"Sept_{var}mnth_avg"] = mnth_avg(df, dss_names, var, 9, units) 

    if var in ["S_SHSTA_", "S_OROVL_"]:
        metrics[f"Apr_{var}mnth_avg"] = mnth_avg(df, dss_names, var, 4, units) 

    ## I'M VERY CONFUSED ABOUT WHAT TO DO FOR THIS ONE. S_SHSTA, S_OROVL, S_TRNTY, S_MELON, S_MLRTN; dead pool set in *LEVEL1DV variables probabilities.
    ## What do I put as the percentile? I put 1. Not returning probability.
    if var in ["S_SHSTA_", "S_OROVL_", "S_TRNTY_", "S_MELON_", "S_MLRTN_"]:
        metrics[f"Sept_{var}mnth_percentile"] = mnth_percentile(df, dss_names, 0.9, var, 9, units)

    ## What do I put as the percentile? I put 1. Not returning probability.
    if var in ["S_SHSTA_"]:
        metrics[f"Apr_{var}mnth_percentile"] = mnth_percentile(df, dss_names, 0.9, var, 4, units)

    ## LONG-TERM CUMULATIVE FRACTION? What is this? S_SHSTA_ & S_SHSTALEVEL5DV

    if var in ["DEL_SWP_TOTAL_", "DEL_SWP_PMI_", "DEL_SWP_PAG_", "DEL_CVP_TOTAL_", "DEL_CVP_PAG_TOTAL_", "DEL_CVP_PSCEX_TOTAL_", "DEL_CVP_PRF_TOTAL_"]:
        metrics[f"{var}ann_avg"] = ann_avg(df, dss_names, var, units)
     
    ## ADD DEL_CVP_PAG_TOTAL +  DEL_CVP_PSCEX_TOTAL

    ## I DON'T SEE VARIABLE D_TOTAL
    ## LONG-TERM AVERAGE? What's the difference between that and average?
    if var in ["NDO_"]: # missing D_TOTAL
        metrics[f"{var}ann_avg"] = ann_avg(df, dss_names, var, units)

    if var in ["X2_PRV_KM_"]:
        metrics[f"Apr_{var}mnth_avg"] = mnth_avg(df, dss_names, var, 4, units) 
        metrics[f"Oct_{var}mnth_avg"] = mnth_avg(df, dss_names, var, 10, units) 
    ## NEED TO ADD WAY TO DO MULTI-MONTH 
        ## Sept-Nov
        ## March-May

    if var in ["DEL_SWP_PMI_", "DEL_CVP_PSC_N_", "DEL_CVP_PAG_N_", "DEL_CVP_PAG_S_", "DEL_CVP_PRF_S_", "DEL_CVP_PMI_N_", "DEL_CVP_PMI_S_"]:
        metrics[f"{var}ann_percentile"] = ann_percentile(df, dss_names, 0.9, var, units)
        
for var, metric_df in metrics.items():
    write_out_dfs.append(metric_df)
    display(metric_df)

,Apr_Avg_S_RESTOT_NOD_TAF
s0001,9502.720622
s0002,9289.524626
s0003,8831.470114
s0004,8536.757009
s0005,8363.576594
s0006,8797.665441
s0007,8818.561157
s0008,8817.579762
s0009,8806.576365
s0010,8809.165562


,Sep_Avg_S_RESTOT_NOD_TAF
s0001,6800.449020
s0002,6570.594230
s0003,5655.789429
s0004,5292.440964
s0005,5134.175172
s0006,6152.457555
s0007,6193.778243
s0008,6194.621633
s0009,6187.818817
s0010,6188.742741


,Apr_Avg_S_SHSTA_TAF
s0001,4037.991051
s0002,3920.254867
s0003,3711.537404
s0004,3592.096895
s0005,3511.278046
s0006,4035.944251
s0007,4044.659226
s0008,4044.682619
s0009,4038.165765
s0010,4038.121174


,Sep_Percentile_S_SHSTA_TAF
s0001,3400.00000
s0002,3400.00000
s0003,3400.00000
s0004,3400.00000
s0005,3400.00000
s0006,3400.00000
s0007,3400.00000
s0008,3400.00000
s0009,3400.00000
s0010,3400.00000


,Apr_Percentile_S_SHSTA_TAF
s0001,4552.10000
s0002,4552.10000
s0003,4495.94533
s0004,4460.78157
s0005,4437.52399
s0006,4552.10000
s0007,4552.10000
s0008,4552.10000
s0009,4552.10000
s0010,4552.10000


...

,Percentile_DEL_CVP_PAG_N_TAF
s0001,34.847494
s0002,34.393217
s0003,33.918754
s0004,33.893795
s0005,33.723046
s0006,34.344864
s0007,34.336341
s0008,34.340126
s0009,34.336667
s0010,34.336660


,Percentile_DEL_CVP_PAG_S_TAF
s0001,143.389351
s0002,143.245424
s0003,138.804389
s0004,136.779033
s0005,135.285446
s0006,139.563284
s0007,142.365815
s0008,144.623838
s0009,144.630669
s0010,144.629032


,Percentile_DEL_CVP_PRF_S_TAF
s0001,23.905811
s0002,23.882565
s0003,23.922530
s0004,23.902585
s0005,23.890469
s0006,24.375671
s0007,24.375671
s0008,24.375671
s0009,24.375671
s0010,24.375671


,Percentile_DEL_CVP_PMI_N_TAF
s0001,1.298429
s0002,13.084345
s0003,1.298429
s0004,13.623956
s0005,1.381785
s0006,14.968287
s0007,1.381785
s0008,14.826032
s0009,1.381785
s0010,15.403063


,Percentile_DEL_CVP_PMI_S_TAF
s0001,13.722242
s0002,13.714244
s0003,13.691231
s0004,13.596215
s0005,13.498519
s0006,13.759796
s0007,13.768386
s0008,13.779971
s0009,13.779993
s0010,13.779988


## Write out dataframes to single CSV

In [19]:
combined_df = pd.concat(write_out_dfs, axis=1)
display(combined_df)
combined_df.to_csv(metrics_path + "/all_metrics_output.csv")

,All_Prob_S_SHSTA_flood,All_Prob_S_SHSTA_dead,Sept_Prob_S_SHSTA_dead,All_Prob_S_OROVL_flood,All_Prob_S_OROVL_dead,Sept_Prob_S_OROVL_dead,All_Prob_S_TRNTY_flood,All_Prob_S_TRNTY_dead,Sept_Prob_S_TRNTY_dead,All_Prob_S_SLUIS_CVP_flood,...,Ann_Avg_DEL_CVP_PRF_TOTAL_TAF,Ann_Avg_NDO_CFS,Apr_Avg_X2_PRV_KM_KM,Oct_Avg_X2_PRV_KM_KM,Percentile_DEL_CVP_PSC_N_TAF,Percentile_DEL_CVP_PAG_N_TAF,Percentile_DEL_CVP_PAG_S_TAF,Percentile_DEL_CVP_PRF_S_TAF,Percentile_DEL_CVP_PMI_N_TAF,Percentile_DEL_CVP_PMI_S_TAF
s0001,0.333333,0.004167,0.01,0.215000,0.000000,0.00,0.000000,0.000000,0.00,0.078333,...,31.060757,21293.470789,64.698729,83.347450,139.458760,34.847494,143.389351,23.905811,1.298429,13.722242
s0002,0.315000,0.017500,0.05,0.210000,0.000833,0.00,0.000833,0.000000,0.00,0.076667,...,30.745665,21893.727002,64.260228,83.458366,139.547772,34.393217,143.245424,23.882565,13.084345,13.714244
s0003,0.222500,0.038333,0.10,0.165833,0.010000,0.02,0.000000,0.000000,0.00,0.055000,...,30.302301,22047.182931,63.974278,84.064856,142.136584,33.918754,138.804389,23.922530,1.298429,13.691231
s0004,0.192500,0.052500,0.11,0.144167,0.016667,0.04,0.000000,0.000000,0.00,0.046667,...,29.720522,21389.026896,64.509556,84.003035,142.706400,33.893795,136.779033,23.902585,13.623956,13.596215
s0005,0.175833,0.053333,0.11,0.133333,0.024167,0.06,0.000000,0.000000,0.00,0.041667,...,29.440866,20485.437723,65.036219,84.739167,143.074190,33.723046,135.285446,23.890469,1.381785,13.498519
s0006,0.303333,0.003333,0.01,0.203333,0.000000,0.00,0.000000,0.004167,0.02,0.085000,...,23.044935,21149.723018,64.831960,83.037519,131.724690,34.344864,139.563284,24.375671,14.968287,13.759796
s0007,0.301667,0.000000,0.00,0.201667,0.000000,0.00,0.000000,0.000000,0.00,0.087500,...,23.073480,20541.031775,65.430541,83.360593,131.710230,34.336341,142.365815,24.375671,1.381785,13.768386
s0008,0.300833,0.000000,0.00,0.200000,0.000000,0.00,0.000000,0.000833,0.01,0.088333,...,23.067966,20408.485673,65.572145,83.385424,131.710228,34.340126,144.623838,24.375671,14.826032,13.779971
s0009,0.300833,0.000000,0.00,0.201667,0.000000,0.00,0.000000,0.000833,0.01,0.090833,...,23.068354,20338.511268,65.654014,83.412193,131.710262,34.336667,144.630669,24.375671,1.381785,13.779993
s0010,0.301667,0.000000,0.00,0.201667,0.000000,0.00,0.000000,0.000833,0.01,0.091667,...,23.068284,20332.168442,65.653841,83.418131,131.710098,34.336660,144.629032,24.375671,15.403063,13.779988
